![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [24]:
# Import necessary libraries
import pandas as pd
import numpy as np 

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()
ds_jobs["company_type"].value_counts()

Pvt Ltd                9817
Funded Startup         1001
Public Sector           955
Early Stage Startup     603
NGO                     521
Other                   121
Name: company_type, dtype: int64

In [25]:
# Create a copy of ds_jobs for transformingz
dictionary = {"gender" : "category","student_id" : "int32", "training_hours" : "int32", "city_development_index": "float16", "company_size": "category", "city": "category", "enrolled_university": "category", "education_level": "category", "major_discipline": "category", "experience" : "category", "company_type": "category","last_new_job": "category"}
ds_jobs_transformed = pd.read_csv("customer_train.csv", dtype = dictionary)

ds_jobs_transformed["relevant_experience"] =  np.where(ds_jobs_transformed["relevant_experience"].str.contains("Has relevant experience"), True , False)
ds_jobs_transformed["job_change"] =  np.where(ds_jobs_transformed["job_change"] == 1, True , False)

ds_jobs_transformed.dtypes


# Start coding here. Use as many cells as you like!

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

In [26]:
print(ds_jobs_transformed["education_level"].value_counts())
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].cat.reorder_categories(new_categories = ["Primary School", "High School", "Graduate","Masters", "Phd"], ordered = True)
print(ds_jobs_transformed["experience"].value_counts())
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].cat.reorder_categories(new_categories = ["<1", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20",">20"], ordered = True)
print(ds_jobs_transformed["company_size"].value_counts())
ds_jobs_transformed["company_size"]= ds_jobs_transformed["company_size"].cat.reorder_categories(new_categories = ["<10", "10-49", "50-99", "100-499","500-999", "1000-4999","5000-9999", "10000+"], ordered = True)
print(ds_jobs_transformed["enrolled_university"].value_counts())
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].cat.reorder_categories(new_categories = ["no_enrollment", "Part time course", "Full time course"], ordered = True)
print(ds_jobs_transformed["last_new_job"].value_counts())
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"] .cat.reorder_categories(new_categories = ["never", "1", "2", "3", "4",">4"], ordered = True)

                                                      
                                                           

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64
1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: last_new_job, dtype: int64


In [27]:
# First, we need to handle NaN values before converting the columns to integers
# Now we can filter the DataFrame
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed["experience"].str.contains("10|11|12|13|14|15|16|17|18|19|20")) & 
    (ds_jobs_transformed["company_size"].str.contains("1000-4999|5000-9999|10000+"))
]
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False
